In [12]:
### Reloads modules properly
%load_ext autoreload
%autoreload 2

In [10]:
%load_ext lab_black

# Figure out what is wrong with the prefix sum

It seems that in certain events the prefix sum is just giving out wrong estimates. Maybe it is actually happening all around but we haven't noticed it.

In [2]:
import numpy as np
import pandas as pd
import math

In [23]:
def prefix_sum(arr):
    """
    Calculates the prefix sum of pT.
    Warning, requires array to be of size thats log base of 2.
    """
    size_log2 = int(np.log2(arr.shape[0]))

    # up-sweep
    for d in range(0, size_log2, 1):
        step_size = 2**d
        double_step_size = step_size * 2

        for i in range(0, arr.shape[0], double_step_size):
            arr[i + double_step_size - 1] += arr[i + step_size - 1]

    # down-sweep
    arr[arr.shape[0] - 1] = 0
    d = size_log2 - 1

    while d >= 0:
        step_size = 2**d
        double_step_size = step_size * 2
        for i in range(0, arr.shape[0], double_step_size):
            tmp = arr[i + step_size - 1]
            arr[i + step_size - 1] = arr[i + double_step_size - 1]
            arr[i + double_step_size - 1] += tmp
        d -= 1

    return arr

In [4]:
arr = np.array([0.39, 0.15, 0.43234, 2.0235])

In [7]:
prefix_sum(arr)

In [8]:
arr

array([0.     , 0.39   , 0.54   , 0.97234])

This gives me the same as the C++ version!

lets load up the data

In [32]:
results

,pv_mc,skl_z0,acc_z0,py_z0,skl_pt,acc_pt,py_pt,passed_skl,passed_acc,passed_py,py2_z0,py2_pt
0,5.055163,4.980469,4.98047,2.460938,84.025032,84.2682,2.460938,True,True,False,2.460938,85.212932
1,-3.608661,-2.226562,-2.22656,-2.226562,142.023392,142.435,-2.226562,False,False,False,-2.226562,125.927683
2,-0.261453,-0.175781,-0.175781,-0.175781,322.723816,323.658,-0.175781,True,True,True,-0.175781,233.015672
3,2.803285,2.753906,2.75391,2.753906,315.833466,316.748,2.753906,True,True,True,2.753906,117.907624
4,-3.764995,-3.837891,-3.80859,-3.837891,241.590347,242.29,-3.837891,True,True,True,-3.808594,137.325767
...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.949224,-1.992188,-1.99219,-1.992188,146.755081,147.18,-1.992188,True,True,True,-1.992188,83.840155
96,-0.863506,-0.820312,-0.820312,-0.820312,261.312958,262.069,-0.820312,True,True,True,-0.820312,173.664954
97,1.698641,1.699219,1.69922,1.699219,315.512695,316.426,1.699219,True,True,True,1.699219,236.707262
98,0.128382,0.175781,0.175781,1.523438,141.515457,141.925,1.523438,True,True,False,1.523438,136.140112


### loading up the data

In [11]:
storage_path = "/media/lucas/QS/binaries-trk-100/"

In [13]:
max_number_of_tracks = 232
eps = 0.15
z0_file = storage_path + "b-0-trk-z0.bin"
pt_file = storage_path + "b-0-trk-pt.bin"
z0 = np.fromfile(z0_file, dtype=np.float32)
pt = np.fromfile(pt_file, dtype=np.float32)

pt needs to be padded to 256 tracks

In [15]:
max_number_of_tracks_power_2 = 1 << (max_number_of_tracks - 1).bit_length()

In [16]:
print(max_number_of_tracks_power_2)

256


In [31]:
n_pad_rs = max_number_of_tracks_power_2 - pt.shape[0]

to_pad_rs = np.zeros(n_pad_rs)

In [44]:
rs = pt.copy()
rs = np.append(rs, to_pad_rs)

In [33]:
pd.set_option("display.max_rows", None)

In [46]:
pt

array([ 2.6239672,  3.4350262,  3.774908 ,  2.6414478,  2.3206532,
        2.7351577,  2.3191988,  2.910093 ,  5.7021084,  3.20127  ,
        2.752108 ,  3.199425 ,  3.1273253,  2.3586159,  2.9780056,
        2.4250774,  2.077471 ,  3.3051517,  5.545457 ,  3.1575665,
        3.6628191,  4.6413064,  3.1045876,  3.5824475,  4.553735 ,
        3.7544825,  2.6030493,  3.7787628,  4.49292  ,  1.9597538,
        2.4486115,  2.2606404,  2.7713442,  3.0753477,  4.8714366,
        4.105771 ,  3.9806397,  2.8278158,  1.9687897,  3.0341637,
        2.0528855,  2.2212894,  3.4607246,  3.1045876,  4.2245073,
        2.6623516,  2.335789 ,  2.4632804,  3.3239539,  2.0978847,
        2.5842652,  2.1006632,  4.01374  ,  2.2510147,  2.3148468,
        3.0341637,  2.014883 ,  2.3090692,  2.185434 ,  2.635178 ,
        2.1953738,  2.5370212,  2.0148828,  3.6483748,  2.6661875,
        3.5917194,  2.812058 ,  2.1490524,  3.2395697,  2.8866367,
        2.332354 ,  2.9973018,  4.3605676,  2.602439 ,  3.2500

In [34]:
rs.shape

(256,)

In [35]:
pt.shape

(159,)

In [36]:
rs = prefix_sum(rs)

In [37]:
rs.shape

(256,)

### C++ results

In [39]:
storage1 = "/media/lucas/QS/accelerated_results_file0/"

In [47]:
input_tracks = pd.read_csv(storage1 + "input_tracks.csv")

In [50]:
input_tracks["pt"] = 0.3 * 3.811 / (input_tracks[" 1/R"] * 100)

In [52]:
input_tracks.sort_values(by="z0")

,z0,1/R,pt
67,-7.441410,0.005305,2.155275
140,-7.441410,0.004430,2.580935
76,-6.972660,0.003459,3.304880
153,-6.562500,0.003459,3.304880
119,-6.503910,0.003444,3.319657
57,-6.328120,0.004937,2.315755
135,-6.328120,0.005062,2.258446
62,-6.269530,0.005658,2.020714
43,-6.152340,0.003672,3.113571
147,-6.035160,0.005757,1.985758


In [40]:
rs_cpp = pd.read_csv(storage1 + "rs.csv")

In [41]:
rs_cpp.head()

,pt_sum
0,0.00000
1,2.58093
2,4.73621
3,8.04109
4,11.34600


### comparison

In [42]:
c1 = pd.DataFrame({})
c1["cpp"] = rs_cpp["pt_sum"]
c1["py"] = rs

In [43]:
c1

,cpp,py
0,0.00000,0.000000
1,2.58093,2.623967
2,4.73621,6.058993
3,8.04109,9.833901
4,11.34600,12.475349
5,14.66560,14.796002
6,16.92410,17.531160
7,19.23980,19.850359
8,21.26050,22.760452
9,24.37410,28.462560


### function re-write?

I think its actually that the tracks are not sorted.

In [54]:
df = pd.DataFrame({"z0": z0, "pt": pt})

In [56]:
df.sort_values(by="z0", inplace=True)

In [58]:
df.reset_index(drop=True, inplace=True)

In [68]:
input_tracks.sort_values(by="z0", inplace=True)

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [69]:
input_tracks.reset_index(drop=True, inplace=True)

In [76]:
input_tracks.iloc[159:, 2] = 0

In [77]:
input_tracks

,z0,1/R,pt
0,-7.441410,0.005305,2.155275
1,-7.441410,0.004430,2.580935
2,-6.972660,0.003459,3.304880
3,-6.562500,0.003459,3.304880
4,-6.503910,0.003444,3.319657
5,-6.328120,0.004937,2.315755
6,-6.328120,0.005062,2.258446
7,-6.269530,0.005658,2.020714
8,-6.152340,0.003672,3.113571
9,-6.035160,0.005757,1.985758


lets do a little test

In [81]:
rs_cpp2 = input_tracks["pt"].values

In [83]:
n_pad = 256 - 232

In [85]:
to_pad = np.zeros(n_pad)

In [86]:
rs_cpp2 = np.append(rs_cpp2, to_pad)

In [87]:
rs_cpp2.shape

(256,)

In [88]:
rs_cpp2 = prefix_sum(rs_cpp2)

In [89]:
rs_cpp2

array([  0.        ,   2.1552748 ,   4.73620978,   8.04108949,
        11.3459692 ,  14.66562669,  16.98138205,  19.23982826,
        21.26054266,  24.37411323,  26.35987094,  31.24541344,
        33.35215657,  37.14185844,  41.18036074,  43.49515024,
        45.85108542,  47.81651125,  50.12602159,  52.8416663 ,
        55.34822703,  57.54995483,  59.97094906,  62.20672102,
        64.62403485,  66.68362293,  69.2954294 ,  71.74518824,
        77.46380413,  82.11855537,  84.31550658,  86.98941093,
        89.07250512,  93.19016065,  95.38624255,  98.20644102,
       100.54899047, 103.09335495, 105.66475268, 108.77832325,
       113.15151113, 115.36461827, 117.70372178, 119.94219023,
       124.47379015, 126.66555259, 129.67153142, 132.28150149,
       135.26812399, 137.27066724, 139.28334572, 141.49206197,
       143.72828435, 145.74899875, 150.25491639, 153.42161906,
       155.94522057, 158.84021529, 161.97659127, 164.65822235,
       167.57673973, 169.5435552 , 171.7575409 , 174.21

In [92]:
rs_cpp2

array([  0.        ,   2.1552748 ,   4.73620978,   8.04108949,
        11.3459692 ,  14.66562669,  16.98138205,  19.23982826,
        21.26054266,  24.37411323,  26.35987094,  31.24541344,
        33.35215657,  37.14185844,  41.18036074,  43.49515024,
        45.85108542,  47.81651125,  50.12602159,  52.8416663 ,
        55.34822703,  57.54995483,  59.97094906,  62.20672102,
        64.62403485,  66.68362293,  69.2954294 ,  71.74518824,
        77.46380413,  82.11855537,  84.31550658,  86.98941093,
        89.07250512,  93.19016065,  95.38624255,  98.20644102,
       100.54899047, 103.09335495, 105.66475268, 108.77832325,
       113.15151113, 115.36461827, 117.70372178, 119.94219023,
       124.47379015, 126.66555259, 129.67153142, 132.28150149,
       135.26812399, 137.27066724, 139.28334572, 141.49206197,
       143.72828435, 145.74899875, 150.25491639, 153.42161906,
       155.94522057, 158.84021529, 161.97659127, 164.65822235,
       167.57673973, 169.5435552 , 171.7575409 , 174.21

In [90]:
rs_cpp.shape

(256, 1)

In [91]:
rs_cpp

,pt_sum
0,0.00000
1,2.58093
2,4.73621
3,8.04109
4,11.34600
5,14.66560
6,16.92410
7,19.23980
8,21.26050
9,24.37410


In [96]:
tracks = input_tracks[["z0", "pt"]].values

In [98]:
z0_pad = np.ones(2)
pt_pad = np.ones(2)

tracks[:,0] = np.append(tracks[:,0], z0_pad)

ValueError: could not broadcast input array from shape (234,) into shape (232,)